# [Project Euler, first problem](https://projecteuler.net/problem=1)

## Multiples of 3 and 5

> If we list all the natural numbers below 10 that are multiples of 3 or 5, we get 3, 5, 6 and 9. The sum of these multiples is 23.

> Find the sum of all the multiples of 3 or 5 below 1000.

Let's create a predicate that returns `True` if a number is a multiple of 3 or 5 and `False` otherwise.  It is simple enough using the modulus operator.

In [1]:
[1 2 3 4 5 6 10 15] [3 % not] map

[false false true false false true false true]

In [2]:
clear

In [3]:
[1 2 3 4 5 6 10 15] [5 % not] map

[false false false false true false true true]

In [4]:
clear

Now we can run them both using `fork` and then `or` the results:

In [5]:
23 [3 % not] [5 % not] fork or

23 false

In [6]:
clear

In [7]:
[1 2 3 4 5 6 10 15] [[3 % not] [5 % not] fork or] map

[false false true false true true true true]

In [8]:
clear

Given the predicate function (and a `filter` function, to be defined later) a suitable program is:

    1000 range [[3 % not] [5 % not] fork or] filter sum

This function generates a list of the integers from 0 to 999, filters that list by the predicate, and then sums the result.

Logically this is fine, but pragmatically we are doing more work than we should.  We generate one thousand integers but actually use less than half of them.  A better solution would be to generate just the multiples we want to sum, and to add them as we go rather than storing them and adding summing them at the end.

Consider the first few terms in the series:

    3 5 6 9 10 12 15 18 20 21 ...

Subtract each number from the one after it (subtracting 0 from 3):

    3 5 6 9 10 12 15 18 20 21 24 25 27 30 ...
    0 3 5 6  9 10 12 15 18 20 21 24 25 27 ...
    -------------------------------------------
    3 2 1 3  1  2  3  3  2  1  3  1  2  3 ...

You get this lovely repeating palindromic sequence:

    3 2 1 3 1 2 3

To make a counter that increments by factors of 3 and 5 you just add
these differences to the counter one-by-one in a loop.

To make use of this sequence to increment a counter and sum terms as we go we need a function that will accept the sum, the counter, and the next term to add, and that adds the term to the counter and a copy of the counter to the running sum.  This function will do that:

    + [+] dupdip

In [9]:
0 0 3 [+ [+] dupdip] trace

  0 0 3 • + [+] dupdip
    0 3 • [+] dupdip
0 3 [+] • dupdip
    0 3 • + 3
      3 • 3
    3 3 • 

3 3

In [10]:
clear

In [11]:
0 0 [3 2 1 3 1 2 3] [+ [+] dupdip] step

60 15

So one `step` through all seven terms brings the counter to 15 and the total to 60.

In [12]:
clear 0 0

0 0

In [13]:
[3 2 1 3 1 2 3] [+ [+] dupdip] step

60 15

In [14]:
[3 2 1 3 1 2 3] [+ [+] dupdip] step

225 30

In [15]:
[3 2 1 3 1 2 3] [+ [+] dupdip] step

495 45

In [16]:
[3 2 1 3 1 2 3] [+ [+] dupdip] step

870 60

Going through one sequence of the palindrome counts off 15 of the 1000.  So how many "flights" in total do we need?

In [17]:
clear

In [18]:
1000 15 /

66

So sixty-six times and a few left over.  How many?

In [19]:
15 *

990

In [20]:
[1000 swap -] trace

     990 • 1000 swap -
990 1000 • swap -
1000 990 • -
      10 • 

10

We only want the terms *less than* 1000.

In [21]:
clear 999 990 -

9

In [22]:
[3 2 1 3] sum

9 9

In [23]:
=

true

That means we want to run the full list of numbers sixty-six times to get to 990 and then the first four numbers 3 2 1 3 to get to 999.

In [24]:
clear 0 0

0 0

In [25]:
66 [[3 2 1 3 1 2 3] [+ [+] dupdip] step] times

229185 990

In [26]:
[3 2 1 3] [+ [+] dupdip] step

233168 999

In [27]:
pop

233168

Ta-da!

## Wee Hack

This form uses no extra storage and produces no unused summands.  It's good but there's one more trick we can apply.  The list of seven terms takes up at least seven bytes.  But notice that all of the terms are less than four, and so each can fit in just two bits.  We could store all seven terms in just fourteen bits and use masking and shifts to pick out each term as we go.  This will use less space and save time loading whole integer terms from the list.

        3  2  1  3  1  2  3
    0b 11 10 01 11 01 10 11 == 14811

In [28]:
clear 14811

14811

In [29]:
[3 &] [2 >>] cleave

3 3702

In [30]:
[3 &] [2 >>] cleave

3 2 925

In [31]:
[3 &] [2 >>] cleave

3 2 1 231

In [32]:
[3 &] [2 >>] cleave

3 2 1 3 57

In [33]:
[3 &] [2 >>] cleave

3 2 1 3 1 14

In [34]:
[3 &] [2 >>] cleave

3 2 1 3 1 2 3

In [35]:
[3 &] [2 >>] cleave

3 2 1 3 1 2 3 0

In [36]:
[3 &] [2 >>] cleave

3 2 1 3 1 2 3 0 0

In [37]:
[3 &] [2 >>] cleave

3 2 1 3 1 2 3 0 0 0

We can run it in a `loop`...

In [38]:
clear 14811

14811

In [39]:
? [[3 &] [2 >>] cleave ?] loop pop

3 2 1 3 1 2 3

?

In [40]:
clear 0 0 14811

0 0 14811

In [41]:
[3 &] [2 >>] cleave

0 0 3 3702

In [42]:
[+ [+] dupdip] dip

3 3 3702

In [43]:
[3 &] [2 >>] cleave

3 3 2 925

In [44]:
[+ [+] dupdip] dip

8 5 925

In [45]:
[3 &] [2 >>] cleave [+ [+] dupdip] dip

14 6 231

In [46]:
[3 &] [2 >>] cleave [+ [+] dupdip] dip

23 9 57

In [47]:
[3 &] [2 >>] cleave [+ [+] dupdip] dip

33 10 14

In [48]:
[3 &] [2 >>] cleave [+ [+] dupdip] dip

45 12 3

In [49]:
[3 &] [2 >>] cleave [+ [+] dupdip] dip

60 15 0

In [50]:
clear 0 0

0 0

In [51]:
14811 7 [[3 &] [2 >>] cleave [+ [+] dupdip] dip] times pop

60 15

In [52]:
14811 7 [[3 &] [2 >>] cleave [+ [+] dupdip] dip] times pop

225 30

In [53]:
clear 0 0

0 0

In [54]:
66 [14811 7 [[3 &] [2 >>] cleave [+ [+] dupdip] dip] times pop] times

229185 990

In [55]:
14811 4 [[3 &] [2 >>] cleave [+ [+] dupdip] dip] times pop

233168 999

In [56]:
pop

233168

In [57]:
clear
[0 swap [dup [pop 14811] [] branch [3 &] [2 >>] cleave] dip rest cons]

[0 swap [dup [pop 14811] [] branch [3 &] [2 >>] cleave] dip rest cons]

In [58]:
x

3 [3702 swap [dup [pop 14811] [] branch [3 &] [2 >>] cleave] dip rest cons]

In [59]:
x x x x x x pop

3 2 1 3 1 2 3

In [60]:
clear
7 66 * 4 +

466

In [61]:
clear

In [62]:
clear
[0 swap [dup [pop 14811] [] branch [3 &] [2 >>] cleave] dip rest cons]

[0 swap [dup [pop 14811] [] branch [3 &] [2 >>] cleave] dip rest cons]

In [63]:
466 [x] times pop

3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3 1 2 3 3 2 1 3

In [64]:
enstacken sum

999

In [65]:
clear 0 0
[0 swap [dup [pop 14811] [] branch [3 &] [2 >>] cleave] dip rest cons]

0 0 [0 swap [dup [pop 14811] [] branch [3 &] [2 >>] cleave] dip rest cons]

In [66]:
466 [x [+ [+] dupdip] dip] times popop

233168